In [ ]:
# !wget https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/121.0.6167.85/win32/chrome-win32.zip


In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium
pip install requests
pip install urljoin
pip install chromedriver_autoinstaller
pip install Pillow


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,796 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,343 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security

In [ ]:
import requests
import bs4
import os
import time
import sys
import time
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from PIL import Image

DIV = "d-flex.flex-column.justify-content-center.align-items-center" # div class name for selection
# DIV = "swatch/"

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
def download_file(url, dir):
    print(url)
    os.makedirs(dir, exist_ok=True)
    print(f'{dir} was created')
    # setup chrome options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless') # ensure GUI is off
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # set path to chromedriver as per your configuratlion
    chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(options=chrome_options)  # 把webdriver放在跟py檔同一個料夾, webdriver一定要跟chrome版本一致
    driver.get(url)

    html = driver.page_source
    soup = bs4.BeautifulSoup(html, 'lxml')
    links = soup.find_all('a',{'class':"d-flex flex-column justify-content-center align-items-center"})
    used_lst = list()  # save all image http routes
    used_lst.append("https://fossil.scene7.com/is/image/FossilPartners/FOSSILOUTLET_placeholder_vid?$sfcc_fos_large$")
    counter = 0
    for link in links:
        link = link.get('href')
        link_url = "https://www.fossil.com"+str(link)
        print(link_url)
        counter = html_download(link_url, dir, driver, counter, used_lst)

def html_download(url, dir, driver,counter, used_lst):
    driver.get(url)
    html = driver.page_source
    soup = bs4.BeautifulSoup(html, 'lxml')
    images = soup.find_all('img')
    for img_tag in images:
        counter += 1
        http = str(img_tag.get('src'))
        if http.endswith('.svg') or http in used_lst or not http.endswith('large$'):
            continue
        used_lst.append(http)
        img_url = urljoin(url, http)
        print(img_url)
        img_response = requests.get(img_url)
        file_name = str(counter) + '.jpg'
        img_name = os.path.join(dir, file_name)
        with open(img_name, 'wb') as f:
            f.write(img_response.content)
        print('Download', img_name)
        im = Image.open(img_name)
        im.save(img_name,'JPEG')

    return counter


def write_file(url):
    with open('download_fail.txt', 'a') as f:
        f.write(f'{url}\n')

In [ ]:
input_file = 'fossil.txt'
out_dir = '../drive/MyDrive/Download4/test'

if os.path.isfile('download_fail.txt'):
  os.remove('download_fail.txt')

if not os.path.isdir('../drive/MyDrive/Download4'):
  os.mkdir('../drive/MyDrive/Download4')
with open(input_file, 'r') as f:
  for i, line in enumerate(f):
    line = line.strip()
    url = line
    dir_ = out_dir+ '_' + str(i)
    download_file(url, dir_)

https://www.fossil.com/en-ca/products/weston-sling-bag/SBG1322001.html
../drive/MyDrive/Download4/test_0 was created
https://www.fossil.com/en-ca/products/weston-sling-bag/SBG1322001.html
https://fossil.scene7.com/is/image/FossilPartners/SBG1322001_main?$sfcc_fos_large$
Download ../drive/MyDrive/Download4/test_0/35.jpg
https://fossil.scene7.com/is/image/FossilPartners/SBG1322001_onmodel?$sfcc_onmodel_large$
Download ../drive/MyDrive/Download4/test_0/36.jpg
https://fossil.scene7.com/is/image/FossilPartners/SBG1322001_1?$sfcc_fos_large$
Download ../drive/MyDrive/Download4/test_0/37.jpg
https://fossil.scene7.com/is/image/FossilPartners/SBG1322001_3?$sfcc_fos_large$
Download ../drive/MyDrive/Download4/test_0/38.jpg
https://fossil.scene7.com/is/image/FossilPartners/SBG1322001_9L?$sfcc_lifestyle_large$
Download ../drive/MyDrive/Download4/test_0/39.jpg
https://fossil.scene7.com/is/image/FossilPartners/SBG1322001_9LU?$sfcc_lifestyle_large$
Download ../drive/MyDrive/Download4/test_0/40.jpg
http